### Import models and load data

In [1]:
from torchvision import datasets
import torch
import dlc_practical_prologue as prologue
import matplotlib.pyplot as plt

# Import all models
from BaseNet import *
from ConvNet1 import *
#from NetSharing import *

mini_batch_size = 1000
nb_epochs = 300
nb_runs = 2
eta = 0.001

train_input, train_target, train_classes, test_input, test_target, test_classes = \
    prologue.generate_pair_sets(nb=1000)

C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### 1channel2images

In [2]:
from importlib import reload
import _1channel2images
reload(_1channel2images)
from _1channel2images import *

print("Working with 1channel2images framework, nb_classes = ", nb_classes)

#model = BaseNet1C(nb_classes)
model_1C = ConvNet1_1C(nb_classes)
optimizer_1C = torch.optim.SGD(model_1C.parameters(), lr=eta, momentum=0.95)
test_results_1C = multiple_training_runs(model_1C, 2, optimizer_1C, train_input, train_classes,
                                      test_input, test_target, test_classes, mini_batch_size, nb_epochs)
write_to_csv('1channel2images.csv', model_1C, test_results_1C)

Working with 1channel2images framework, nb_classes =  10


KeyboardInterrupt: 

In [ ]:
# This is just for visualization of our results but it will have to be taken away for the report 
# since we can't use any additional libraries
import pandas as pd

print(test_results_1C)
write_to_csv('1channel2images.csv', model_1C, test_results_1C)
data = pd.read_csv('1channel2images.csv')
data.head()

### 2channels1image

In [ ]:
import _2channels1image
reload(_2channels1image)
from _2channels1image import *

print("Working with 2channels1image framework, nb_classes = ", nb_classes)

model_2C = ConvNet1_2C(nb_classes)
optimizer_2C = torch.optim.SGD(model_2C.parameters(), lr=0.00001, momentum=0.95)  
test_results_2C = multiple_training_runs(model_2C, 2, optimizer_2C, train_input, train_target,
                           test_input, test_target, mini_batch_size, nb_epochs)
write_to_csv('2channels1image.csv', model_2C, test_results_2C)

In [ ]:
print(test_results_2C)
write_to_csv('2channels1image.csv', model_2C, test_results_2C)
data = pd.read_csv('2channels1image.csv')
data.head()

### Weight Sharing

In [ ]:
import weight_sharing
reload(weight_sharing)
from weight_sharing import *

print("Working with weight_sharing framework")

model_ws = NetSharing1()
optimizer_ws = torch.optim.SGD(model_ws.parameters(), lr=0.00001, momentum=0.95)  
test_results_ws = multiple_training_runs(model_ws, nb_runs, optimizer_ws, train_input, train_target,
                           test_input, test_target, mini_batch_size, nb_epochs)
write_to_csv('weightsharing.csv', model_ws, test_results_ws)

In [ ]:
print(test_results_ws)
write_to_csv('weightsharing.csv', model_ws, test_results_ws)
data = pd.read_csv('weightsharing.csv')
data.head()

### Auxiliary Losses

In [ ]:
import auxiliary_losses
reload(auxiliary_losses)
from auxiliary_losses import *
import Incept1
reload(Incept1)
from Incept1 import *
nb_epochs = 300
print("Working with auxiliary losses framework")

model_aux = Incept1()
optimizer_aux = torch.optim.SGD(model_aux.parameters(), lr=0.00001, momentum=0.95)  
test_results_aux = multiple_training_runs(model_aux, nb_runs, optimizer_aux, train_input, train_target, train_classes, 
                                          test_input, test_target, test_classes, mini_batch_size, nb_epochs)
write_to_csv('auxiliary_losses.csv', model_aux, test_results_aux)

In [ ]:
print(test_results_aux)
write_to_csv('auxiliary_losses.csv', model_aux, test_results_aux)
data = pd.read_csv('auxiliary_losses.csv')
data.head()

## Weight Sharing and Auxiliary Loss Models(Youssef)

In [1]:
print(train_input.shape, train_target.shape, train_classes.shape, test_input.shape, test_target.shape, test_classes.shape)

NameError: name 'train_input' is not defined

In [2]:
from torch.autograd import Variable
from WSharingModel import *
from torch import nn, optim
from torch.nn import CrossEntropyLoss
from torch.nn import BCEWithLogitsLoss
import torch
from torchvision import datasets
from torch import optim
import dlc_practical_prologue as prologue
from AuxModel import *
from torch.autograd import Variable
from torch.nn import functional as F
from time import time
from SimpleCNN import *


#### Function that draws the evolution of the loss during training.

In [3]:
def visualize_loss(model,loss_model,lr):
    plt.plot(loss_model[0],loss_model[1], color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Evolution of the loss during training with learning rate {:>5} of the model {:>5}.'.format(lr,model.__class__.__name__))
    plt.show()

#### Function that prints the results of the prediction one the model is trained.

In [4]:
def print_results(model,optimizer,learning_rate,NB_EPOCHS,MINI_BATCH_SIZE, train_input, train_target,test_input, test_target, time_diff):
     print('model: {:6}, optimizer: {:6}, learning rate: {:6}, num epochs: {:3}, '
                    'mini batch size: {:3}, train error: {:5.2f}%, test error: {:5.2f}%, time: {:.3f}'.format(
                    model.__class__.__name__,
                    optimizer.__name__,
                    learning_rate,
                    NB_EPOCHS,
                    MINI_BATCH_SIZE,
                    model.compute_nb_errors(model, train_input, train_target, MINI_BATCH_SIZE) / train_input.size(0) * 100,
                    model.compute_nb_errors(model, test_input, test_target, MINI_BATCH_SIZE) / test_input.size(0) * 100,
                    time_diff
                    )
                )

### Treatement of our Training and Testing Data.

In [5]:
# normalize it
mean, std = train_input.mean(), train_input.std() 
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

train_input, train_target, train_classes = Variable(train_input), Variable(train_target), Variable(train_classes)
test_input, test_target = Variable(test_input), Variable(test_target)

NameError: name 'train_input' is not defined

### We define our Learning parameters.

In [6]:
NB_EPOCHS = 25
MINI_BATCH_SIZE = 100
learning_rates = [0.1, 0.01]

### We define our optimizers and losses and weights for the auxiliary losses.

In [7]:
op = torch.optim.SGD
losses = [CrossEntropyLoss(),BCEWithLogitsLoss()]
alpha = 0.5
beta = 1

### Training of Different Models and Results on Training and Testing set.

In [10]:
 def calculate_results(result_dct):
    k = 0
    for j in range(5):
        for i in range(len(learning_rates)):
            models = [WSModel(),WSModel1(),AuxModel(),AuxModel1(), SimpleModel()]
            model = models[j]
            optimizer = op(model.parameters(),lr = learning_rates[i])
            if (j<2):
                #print("here")
                start_time = time()
                loss_aux = train_model_WS(model, optimizer,  train_input, train_target, NB_EPOCHS, MINI_BATCH_SIZE,losses[0])
                end_time = time()
            elif (j==4):
                start_time = time()
                loss_aux = train_model(model, optimizer,  train_input, train_target, NB_EPOCHS, MINI_BATCH_SIZE,losses[0])
                end_time = time()
            elif(j>1):
                #print("here here")
                start_time = time()
                loss_aux = train_model_AM(model, optimizer,  train_input, train_target, train_classes,NB_EPOCHS, MINI_BATCH_SIZE,losses[0],alpha,beta)
                end_time = time()
            #print("model:",model)
            time_diff = end_time - start_time
            print_results(model,op,learning_rates[i],NB_EPOCHS,MINI_BATCH_SIZE, train_input, train_target,test_input, test_target, time_diff)
            print(result_dct[k])
            train_error = model.compute_nb_errors(model, train_input, train_target, MINI_BATCH_SIZE) / train_input.size(0) * 100
            test_error = model.compute_nb_errors(model, test_input, test_target, MINI_BATCH_SIZE) / test_input.size(0) * 100
            result_dct[k]['train_error'].append(train_error)
            result_dct[k]['test_error'].append(test_error)
            result_dct[k]['time'].append(time_diff)
            k += 1
    return result_dct
            #visualize_loss(model,loss_aux,learning_rates[i])

In [11]:
from collections import defaultdict
result_dct = defaultdict(dict)
for i in range(10):
    result_dct[i]['train_error'] = []
    result_dct[i]['test_error'] = []
    result_dct[i]['time'] = []
for i in range(10):
    
    train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(nb=1000)
    # normalize it
    mean, std = train_input.mean(), train_input.std() 
    train_input.sub_(mean).div_(std)
    test_input.sub_(mean).div_(std)

    train_input, train_target, train_classes = Variable(train_input), Variable(train_target), Variable(train_classes)
    test_input, test_target = Variable(test_input), Variable(test_target)
    
    print(result_dct)
    result_dct = calculate_results(result_dct)

C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


defaultdict(<class 'dict'>, {0: {'train_error': [], 'test_error': [], 'time': []}, 1: {'train_error': [], 'test_error': [], 'time': []}, 2: {'train_error': [], 'test_error': [], 'time': []}, 3: {'train_error': [], 'test_error': [], 'time': []}, 4: {'train_error': [], 'test_error': [], 'time': []}, 5: {'train_error': [], 'test_error': [], 'time': []}, 6: {'train_error': [], 'test_error': [], 'time': []}, 7: {'train_error': [], 'test_error': [], 'time': []}, 8: {'train_error': [], 'test_error': [], 'time': []}, 9: {'train_error': [], 'test_error': [], 'time': []}})
Loss at   0 : 0.6575838327407837  
model: WSModel, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  0.20%, test error: 13.40%, time: 43.813
{'train_error': [], 'test_error': [], 'time': []}
Loss at   0 : 0.6796262264251709  
model: WSModel, optimizer: SGD   , learning rate:   0.01, num epochs:  25, mini batch size: 100, train error: 19.90%, test error: 20.10%, time: 72.799
{'train_

model: WSModel, optimizer: SGD   , learning rate:   0.01, num epochs:  25, mini batch size: 100, train error: 20.90%, test error: 22.30%, time: 39.740
{'train_error': [19.900000000000002, 21.5], 'test_error': [20.1, 21.7], 'time': [72.79865550994873, 53.022902727127075]}
Loss at   0 : 0.6652682423591614  
model: WSModel1, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  2.20%, test error: 16.50%, time: 39.808
{'train_error': [0.4, 19.6], 'test_error': [13.200000000000001, 24.4], 'time': [46.990843772888184, 34.247894287109375]}
Loss at   0 : 0.6839495897293091  
model: WSModel1, optimizer: SGD   , learning rate:   0.01, num epochs:  25, mini batch size: 100, train error: 21.70%, test error: 23.30%, time: 40.069
{'train_error': [21.5, 20.8], 'test_error': [21.2, 21.0], 'time': [51.6102774143219, 34.75338888168335]}
Loss at   0 : 4.920703887939453  
model: AuxModel, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 1

defaultdict(<class 'dict'>, {0: {'train_error': [0.2, 0.2, 2.8000000000000003, 1.0], 'test_error': [13.4, 14.6, 17.1, 13.200000000000001], 'time': [43.813161849975586, 61.13934254646301, 33.61184215545654, 38.24961495399475]}, 1: {'train_error': [19.900000000000002, 21.5, 20.9, 18.2], 'test_error': [20.1, 21.7, 22.3, 22.3], 'time': [72.79865550994873, 53.022902727127075, 39.74032783508301, 43.013503074645996]}, 2: {'train_error': [0.4, 19.6, 2.1999999999999997, 0.2], 'test_error': [13.200000000000001, 24.4, 16.5, 12.9], 'time': [46.990843772888184, 34.247894287109375, 39.80758619308472, 42.55494976043701]}, 3: {'train_error': [21.5, 20.8, 21.7, 17.9], 'test_error': [21.2, 21.0, 23.3, 20.8], 'time': [51.6102774143219, 34.75338888168335, 40.069377422332764, 50.13468933105469]}, 4: {'train_error': [6.2, 16.8, 27.200000000000003, 7.9], 'test_error': [10.2, 19.5, 26.200000000000003, 11.3], 'time': [50.46531391143799, 35.45557117462158, 41.461453914642334, 38.552266120910645]}, 5: {'train_er

model: WSModel, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  3.20%, test error: 13.20%, time: 35.307
{'train_error': [0.2, 0.2, 2.8000000000000003, 1.0, 1.0999999999999999], 'test_error': [13.4, 14.6, 17.1, 13.200000000000001, 14.799999999999999], 'time': [43.813161849975586, 61.13934254646301, 33.61184215545654, 38.24961495399475, 32.15898060798645]}
Loss at   0 : 0.689903736114502  
model: WSModel, optimizer: SGD   , learning rate:   0.01, num epochs:  25, mini batch size: 100, train error: 22.30%, test error: 22.80%, time: 35.126
{'train_error': [19.900000000000002, 21.5, 20.9, 18.2, 19.5], 'test_error': [20.1, 21.7, 22.3, 22.3, 22.0], 'time': [72.79865550994873, 53.022902727127075, 39.74032783508301, 43.013503074645996, 33.48177671432495]}
Loss at   0 : 0.6831420660018921  
model: WSModel1, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  3.20%, test error: 14.40%, time: 35.093
{'train_e

{'train_error': [0.4, 19.6, 2.1999999999999997, 0.2, 0.4, 3.2], 'test_error': [13.200000000000001, 24.4, 16.5, 12.9, 15.6, 14.399999999999999], 'time': [46.990843772888184, 34.247894287109375, 39.80758619308472, 42.55494976043701, 33.975629568099976, 35.0931339263916]}
Loss at   0 : 0.6891385912895203  
model: WSModel1, optimizer: SGD   , learning rate:   0.01, num epochs:  25, mini batch size: 100, train error: 21.50%, test error: 23.20%, time: 33.670
{'train_error': [21.5, 20.8, 21.7, 17.9, 20.4, 19.6], 'test_error': [21.2, 21.0, 23.3, 20.8, 20.9, 21.9], 'time': [51.6102774143219, 34.75338888168335, 40.069377422332764, 50.13468933105469, 33.78319764137268, 36.62995767593384]}
Loss at   0 : 4.877391338348389  
model: AuxModel, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  5.40%, test error: 10.50%, time: 33.148
{'train_error': [6.2, 16.8, 27.200000000000003, 7.9, 6.800000000000001, 7.6], 'test_error': [10.2, 19.5, 26.200000000000003, 11

model: WSModel1, optimizer: SGD   , learning rate:   0.01, num epochs:  25, mini batch size: 100, train error: 23.80%, test error: 24.10%, time: 33.682
{'train_error': [21.5, 20.8, 21.7, 17.9, 20.4, 19.6, 21.5], 'test_error': [21.2, 21.0, 23.3, 20.8, 20.9, 21.9, 23.200000000000003], 'time': [51.6102774143219, 34.75338888168335, 40.069377422332764, 50.13468933105469, 33.78319764137268, 36.62995767593384, 33.67032599449158]}
Loss at   0 : 4.925912380218506  
model: AuxModel, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  9.60%, test error: 15.00%, time: 44.522
{'train_error': [6.2, 16.8, 27.200000000000003, 7.9, 6.800000000000001, 7.6, 5.4], 'test_error': [10.2, 19.5, 26.200000000000003, 11.3, 11.5, 11.600000000000001, 10.5], 'time': [50.46531391143799, 35.45557117462158, 41.461453914642334, 38.552266120910645, 28.499269723892212, 33.958043813705444, 33.14839839935303]}
Loss at   0 : 4.963137626647949  
model: AuxModel, optimizer: SGD   , l

model: WSModel1, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  0.40%, test error: 14.80%, time: 32.626
{'train_error': [0.4, 19.6, 2.1999999999999997, 0.2, 0.4, 3.2, 0.7000000000000001, 1.2], 'test_error': [13.200000000000001, 24.4, 16.5, 12.9, 15.6, 14.399999999999999, 15.2, 16.2], 'time': [46.990843772888184, 34.247894287109375, 39.80758619308472, 42.55494976043701, 33.975629568099976, 35.0931339263916, 33.11528134346008, 34.220024824142456]}
Loss at   0 : 0.6881752610206604  
model: WSModel1, optimizer: SGD   , learning rate:   0.01, num epochs:  25, mini batch size: 100, train error: 17.00%, test error: 21.30%, time: 31.935
{'train_error': [21.5, 20.8, 21.7, 17.9, 20.4, 19.6, 21.5, 23.799999999999997], 'test_error': [21.2, 21.0, 23.3, 20.8, 20.9, 21.9, 23.200000000000003, 24.099999999999998], 'time': [51.6102774143219, 34.75338888168335, 40.069377422332764, 50.13468933105469, 33.78319764137268, 36.62995767593384, 33.67032599449158, 3

model: WSModel, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  2.30%, test error: 17.90%, time: 49.831
{'train_error': [0.2, 0.2, 2.8000000000000003, 1.0, 1.0999999999999999, 3.2, 3.1, 1.2, 0.2], 'test_error': [13.4, 14.6, 17.1, 13.200000000000001, 14.799999999999999, 13.200000000000001, 16.900000000000002, 16.3, 16.2], 'time': [43.813161849975586, 61.13934254646301, 33.61184215545654, 38.24961495399475, 32.15898060798645, 35.30684280395508, 32.42444920539856, 32.39015793800354, 35.473501443862915]}
Loss at   0 : 0.706010639667511  
model: WSModel, optimizer: SGD   , learning rate:   0.01, num epochs:  25, mini batch size: 100, train error: 20.60%, test error: 22.90%, time: 49.539
{'train_error': [19.900000000000002, 21.5, 20.9, 18.2, 19.5, 22.3, 20.200000000000003, 24.2, 17.4], 'test_error': [20.1, 21.7, 22.3, 22.3, 22.0, 22.8, 23.7, 25.1, 22.0], 'time': [72.79865550994873, 53.022902727127075, 39.74032783508301, 43.013503074645996, 33.48

0 = WSModel, lr = .1 <br/>
1 = WSModel, lr = 0.01  <br/>
2 = WSModel1, lr = .1 <br/>
3 = WSModel1, lr = 0.01 <br/>
4 = AuxModel, lr = .1 <br/>
5 = AuxModel, lr = 0.01 <br/>
6 = AuxModel1, lr = .1 <br/>
7 = AuxModel1, lr = 0.01 <br/>
8 = Base, lr = .1 <br/>
9 = Base, lr = 0.01 <br/>

In [12]:
import statistics
for k,v in result_dct.items():
    print('mean of train_error of model {} = {}'.format(k, statistics.mean(v['train_error'])))
    print('stdev of train_error of model {} = {}'.format(k, statistics.stdev(v['train_error'])))
    print('mean of test_error of model {} = {}'.format(k, statistics.mean(v['test_error'])))
    print('stdev of test_error of model {} = {}'.format(k, statistics.stdev(v['test_error'])))
    print('mean of time of model {} = {}'.format(k, statistics.mean(v['time'])))
    print('stdev of time of model {} = {}'.format(k, statistics.stdev(v['time'])))
    print()

mean of train_error of model 0 = 1.53
stdev of train_error of model 0 = 1.2175111224680182
mean of test_error of model 0 = 15.36
stdev of test_error of model 0 = 1.7481418706729726
mean of time of model 0 = 39.4398610830307
stdev of time of model 0 = 9.53406190056124

mean of train_error of model 1 = 20.47
stdev of train_error of model 1 = 1.956215166534034
mean of test_error of model 1 = 22.490000000000002
stdev of test_error of model 1 = 1.309325526113859
mean of time of model 1 = 43.07329266071319
stdev of time of model 1 = 12.554349580673703

mean of train_error of model 2 = 2.98
stdev of train_error of model 2 = 5.917356767266202
mean of test_error of model 2 = 16.15
stdev of test_error of model 2 = 3.3026083967407067
mean of time of model 2 = 37.51026966571808
stdev of time of model 2 = 5.035451278157589

mean of train_error of model 3 = 20.54
stdev of train_error of model 3 = 1.9642357858012416
mean of test_error of model 3 = 22.09
stdev of test_error of model 3 = 1.233288287465

In [14]:
for model in [WSModel(), WSModel1(), AuxModel(), AuxModel1(), SimpleModel()]:
    print(len(list(model.parameters())))

10
8
12
10
8
